# Import libraries

In [1]:
from GridSearch import GridSearch
from keras import backend as K 
from keras.engine import Layer
from keras.layers import Dense, Dropout, Embedding, Flatten, Input, MaxPooling1D
from keras.models import Model, load_model, Sequential
from keras.optimizers import Adam, SGD
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasRegressor
from scipy.stats import pearsonr
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier, GradientBoostingRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, MultiTaskElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, LeaveOneOut
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR, LinearSVR
from xgboost import XGBRegressor
import csv
import keras.layers as layers
import nltk
import sklearn
import numpy as np
import os
import pandas as pd
import re
import sys
import xgboost as xgb

Using TensorFlow backend.


In [2]:
os.getcwd()
print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The nltk version is 3.4.5.
The scikit-learn version is 0.22.1.


# GridSearch Object

In [3]:
class GridSearch:

    def __init__(self, models_dict, params_dict):
        # if not set(models_dict.keys()).issubset(set(params_dict.keys())):
        #     missing_params = list(set(models.keys()) - set(params_dict.keys()))
        #     raise ValueError(
        #         "Some estimators are missing parameters: %s" % missing_params)
        self.models = models_dict
        self.params = params_dict
        self.keys = models_dict.keys()
        #print(self.keys)
        self.best_ = {
            'estimator': [None],
            'params': {},
            'y_pred': [],
            'r': [],
        }

    def predict(self):
        return self.best_['r']

    def tune(self, X_train, y_train, X_test, y_test, **grid_kwargs):
        max_r = 0
        for key in self.keys:
            print("\tRunning GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]

            #Pipeline the estimators
            pipeline = Pipeline([
                ('clf', model),
            ])

            gs = GridSearchCV(pipeline, params, **grid_kwargs)
            gs.fit(X_train, y_train)

            print("\tPredicting for %s." % key)
            y_pred = gs.predict(X_test)
            r = np.corrcoef(y_pred, y_test)[0, 1]
            print(params)
            print(r)

            if (abs(r) > abs(max_r)):
                self.best_['estimator'] = model
                self.best_['params'] = gs.best_params_
                self.best_['r'] = r
                self.best_['y_pred'] = y_pred

            print("Current Best")
            #print(self.best_['params'])
            print(self.best_['r'])

            print('\tTuning for %s Done.' % key)

In [4]:
def ElmoRegressionModel(
    dense_dropout_rate=0.5,
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mse'],
    print_summary=False,
    include_hidden_layer=False,
    hidden_layer_size=64
):
    inputs, embeddings = [], []
    
    for idx in range(1, 6):
        _input = layers.Input(shape=(1,), dtype="string")
        inputs.append(_input)
        embedding = ElmoEmbeddingLayer()(_input)
        embeddings.append(embedding)
        
    concat = layers.concatenate(embeddings)
    dense = Dropout(dense_dropout_rate)(concat)
    if include_hidden_layer:
        dense = layers.Dense(hidden_layer_size, activation='relu')(dense)
        dense = Dropout(dense_dropout_rate)(dense)
    dense = layers.Dense(1, activation='relu')(dense)# (drop2)
    
    # If we want to do 5-way prediction within a single network
    # dense = layers.Dense(5, activation='relu')(dense)
    
    model = Model(inputs=inputs, outputs=dense)

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    
    if print_summary:
        model.summary()

    return model

In [5]:
###############################################################################################################
# Import the process data from R Studio
###############################################################################################################
data = pd.read_csv("mega_dataset.csv")

train_data = data.loc[data.Dataset == "Train"]
train_data_X = train_data.iloc[:, 13:]
train_data_y = train_data.loc[:, train_data.columns.str.contains('_Scale_score')]

test_data = data.loc[(data.Dataset == "Dev") | (data.Dataset == "Test")]
test_data_X = test_data.iloc[:, 13:]
test_data_y = test_data.loc[:, test_data.columns.str.contains('_Scale_score')]

In [6]:
# best_hyperparameter_list = tune(train_data_X, train_data_y, dev_data_X, dev_data_y, selected_feature)
best_hyperparameters = []
clf_dict = {
    'neural': KerasRegressor(build_fn=lambda: ElmoRegressionModel(**model_params)),
    'forest': RandomForestRegressor(),
    'ridge': Ridge(),
    'elastic': ElasticNet(),
}

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
param_list = {
    'neural': {
        'clf__build_fn': [lambda: ElmoRegressionModel(**model_params)],
        'clf__epochs': [10],
        'clf__batch_size': [32],
        'clf__verbose': [1]
    },
    'forest': {
        'clf__n_estimators': [1000, 1500, 2000, 2500, 3000, 3500, 4000, 5000, 8000, 10000],
        'clf__criterion': ['mse'],
        'clf__max_depth': max_depth,
        'clf__min_samples_split': [2, 3, 4, 5, 6, 10, 12],
        'clf__min_samples_leaf': [1, 2, 4],
        'clf__min_weight_fraction_leaf': [0.0],
        'clf__max_features': ['auto', 'sqrt'],
        'clf__max_leaf_nodes': [None],
        'clf__min_impurity_decrease': [0.0],
        'clf__min_impurity_split': [None],
        'clf__bootstrap': [True, False],
        'clf__oob_score': [False],
        'clf__n_jobs': [None],
        'clf__random_state': [None],
        'clf__verbose': [0],
        'clf__warm_start': [False],
        'clf__ccp_alpha': [0.0],
        'clf__max_samples': [None]
    },
    'ridge': {
        'clf__alpha': [0.0, 0.05, 0.1, 0.3, 0.5, 0.8, 0.9, 0.95, 1.0, 1.2, 1.8, 10, 100, 10000, 1000000], 
        'clf__fit_intercept': [True], 
        'clf__normalize': [False], 
        'clf__copy_X': [True],
        'clf__max_iter': [None],
        'clf__tol': [0.001],
        'clf__solver': ['auto'],
        'clf__random_state': [None],
    },
    'elastic': {
        'clf__alpha': [0.0, 0.05, 0.1, 0.3, 0.5, 0.8, 0.9, 0.95, 1.0, 1.2, 1.8, 10, 100, 10000, 1000000],
        'clf__l1_ratio': [0.0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1.0],
        'clf__fit_intercept': [True],
        'clf__normalize': [False],
        'clf__precompute': [False],
        'clf__max_iter': [1000],
        'clf__copy_X': [True],
        'clf__tol': [0.0001],
        'clf__warm_start': [False],
        'clf__positive': [False],
        'clf__random_state': [None],
        'clf__selection': ['cyclic']
    },
}

In [7]:
Corr = {
    'neural_r2': None,
    'neural_mse': None,
    'forest_r2': None,
    'forest_mse': None,
    'ridge_r2': None,
    'ridge_mse': None,
    'elastic_r2': None,
    'elastic_mse': None,
}

# Run Neural Network

In [ ]:
print('Running Parameter TUning for Neural Network with r2')
pipe = Pipeline([('clf', clf_dict.get("neural"))])
gs = GridSearchCV(pipe, param_list.get("neural"), cv=10, n_jobs=10, verbose=5, scoring='r2', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
neural_y = gs.predict(test_data_X)
Corr['neural_r2'] = forest_y.corr(test_data_y)

In [ ]:
print('Running Parameter TUning for Neural Network with mse')
pipe = Pipeline([('clf', clf_dict.get("neural"))])
gs = GridSearchCV(pipe, param_list.get(key), cv=10, n_jobs=10, verbose=1, scoring='mean_squared_error', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
neural_y = gs.predict(test_data_X)
Corr['neural_mse'] = forest_y.corr(test_data_y)

# Run Random Forest

In [ ]:
print('Running Parameter TUning for Random Forest with r2')
pipe = Pipeline([('clf', clf_dict.get("forest"))])
gs = GridSearchCV(pipe, param_list.get("forest"), cv=10, n_jobs=30, verbose=5, scoring='r2', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
forest_y = gs.predict(test_data_X)
Corr['forest_r2'] = forest_y.corr(test_data_y)
Corr

Running Parameter TUning for Random Forest with r2
Fitting 10 folds for each of 13104 candidates, totalling 131040 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  12 tasks      | elapsed:  9.1min
[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed: 71.7min


In [ ]:
print('Running Parameter TUning for Random Forest with mse')
pipe = Pipeline([('clf', clf_dict.get("neural"))])
gs = GridSearchCV(pipe, param_list.get(key), cv=10, n_jobs=30, verbose=5, scoring='mean_squared_error', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
forest_y = gs.predict(test_data_X)
Corr['forest_mse'] = forest_y.corr(test_data_y)
Corr

# Run Ridge Regression

In [ ]:
print('Running Parameter TUning for Ridge with r2')
pipe = Pipeline([('clf', clf_dict.get("ridge"))])
gs = GridSearchCV(pipe, param_list.get("ridge"), cv=10, n_jobs=30, verbose=5, scoring='r2', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
forest_y = gs.predict(test_data_X)
Corr['ridge_r2'] = forest_y.corr(test_data_y)
Corr

In [ ]:
print('Running Parameter TUning for Ridge with mse')
pipe = Pipeline([('clf', clf_dict.get("ridge"))])
gs = GridSearchCV(pipe, param_list.get("ridge"), cv=10, n_jobs=30, verbose=5, scoring='r2', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
forest_y = gs.predict(test_data_X)
Corr['ridge_mse'] = forest_y.corr(test_data_y)
Corr

# Run Elastic Net

In [ ]:
print('Running Parameter TUning for Elastic Net with r2')
pipe = Pipeline([('clf', clf_dict.get("elastic"))])
gs = GridSearchCV(pipe, param_list.get("elastic"), cv=10, n_jobs=30, verbose=5, scoring='r2', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
forest_y = gs.predict(test_data_X)
Corr['elastic_r2'] = forest_y.corr(test_data_y)
Corr

In [ ]:
print('Running Parameter TUning for Elastic Net with mse')
pipe = Pipeline([('clf', clf_dict.get("elastic"))])
gs = GridSearchCV(pipe, param_list.get("elastic"), cv=10, n_jobs=30, verbose=5, scoring='r2', return_train_score=False, error_score='raise', iid=True)
gs.fit(train_data_X, train_data_y)
forest_y = gs.predict(test_data_X)
Corr['elastic_mse'] = forest_y.corr(test_data_y)
Corr